# Spearman Correlation
The idea is to analyse the correlation between the disease rates and suicide rates in each municipality.

Import libraries

In [1]:
import pandas as pd
import glob
from scipy.stats import spearmanr
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

root = "../"

C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\pypas\anaconda3\envs\ML\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


Read suicide csv

In [2]:
suicide = pd.read_csv(root + 'CSV/Suicide/suicide_rates_08_18.csv', sep=',', index_col=0)
suicide.head()

MUNCOD    RATE_08    RATE_09    RATE_10   RATE_11    RATE_12    RATE_13  \
0  110001  20.344224   8.212203   8.189337  4.127456  12.464166   7.773632   
1  110002   9.458389   2.338060   4.427031  4.368243   9.703818   1.974938   
2  110003   0.000000  14.936520   0.000000  0.000000   0.000000  15.396459   
3  110004   5.110972   7.626311   2.544497  1.266480   5.042229   1.164646   
4  110005   0.000000   0.000000  11.743981  0.000000  11.868028   0.000000   

    RATE_14    RATE_15    RATE_16    RATE_17    RATE_18  
0  3.898332  11.728829   7.841292  11.793844   4.316485  
1  4.860976   4.789226   5.665936   9.315758   1.883807  
2  0.000000  31.471282  15.900779   0.000000  18.389114  
3  5.776607   6.878683  10.241588   5.649271   9.432516  
4  0.000000  11.119760  11.136478   5.576001   6.081245

In [8]:
disease = ""
path = root + "CSV/TabNet/Internacoes_Rate/"
all_files = glob.glob(path + "*.csv")

years = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
suicide_df = suicide.copy()
suicide_df["SUICIDE"] = suicide_df.drop(columns="MUNCOD").sum(axis=1)/(len(suicide_df.columns) - 1)
suicide_df = suicide_df[["MUNCOD", "SUICIDE"]]

diseases_list = []
corrs_list = []
p_value_list = []
size_list = []

for file in all_files:
    file_name = file.split("\\")[1]
    disease = file_name.split(".csv")[0]
    
    # Load disease file
    disease_df = pd.read_csv(path + disease + '.csv', sep=',', index_col=0) 
    
    # Determine disease average rate
    disease_df["Total"] = disease_df.drop(columns="MUNCOD").sum(axis=1)/(len(disease_df.columns) - 1)
    disease_df = disease_df[["MUNCOD", "Total"]]
    
    # Exclude rows with zero rate of suicides
    disease_df = disease_df[(disease_df["Total"] != 0)] 
    
    # Include only diseases with data of more than half of the municipalities 
    if disease_df.shape[0] > 2785:
        final_df = pd.merge(disease_df, suicide_df, on="MUNCOD")
        size_list.append(final_df.shape[0])
        corr, p_value = spearmanr(final_df["Total"],final_df["SUICIDE"])
        diseases_list.append(disease)
        corrs_list.append(corr)
        p_value_list.append(p_value)

In [9]:
corrs_data = {'Doenças': diseases_list, 'Correlação com suicidio': corrs_list, 'P value': p_value_list, "Qtd Municipios": size_list}
corrs_df = pd.DataFrame(data=corrs_data)
corrs_df.sort_values(by=['Correlação com suicidio'], ascending=False, inplace=True)

In [11]:
corrs_df.reset_index(drop=True, inplace=True)

In [12]:
corrs_df.head(50)

Doenças  \
0                     TRANSTORNOS_DE_HUMOR_[AFETIVOS]   
1   TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁL...   
2   NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES   
3                                             ARTROSE   
4   TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...   
5       TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS   
6                           COLELITÍASE_E_COLECISTITE   
7   TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT   
8                           NEOPLASIA_MALIGNA_DA_PELE   
9                OUTRAS_DOENÇAS_ISQUÊMICAS_DO_CORAÇÃO   
10                         NEOPLASIA_MALIGNA_DO_CÓLON   
11                      NEOPLASIA_MALIGNA_DO_PÂNCREAS   
12    TRANSTORNOS_DOS_NERVOS_RAÍZES_E_PLEXOS_NERVOSOS   
13                       EMBOLIA_E_TROMBOSE_ARTERIAIS   
14  BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC   
15                                   EMBOLIA_PULMONAR   
16                                DOENÇAS_DO_APÊNDICE   
17  NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_E...   
18    FLEBITE_TROMBOFLEBITE_EMBOLIA_E_TROMBOSE_VENOSA   
19                 OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE   
20  NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL   
21                      NEOPLASIA_MALIGNA_DO_ENCÉFALO   
22           DEFORMIDADES_ADQUIRIDAS_DAS_ARTICULAÇÕES   
23  NEOPL_MALIG_OUTRAS_PARTES_SISTEMA_NERVOSO_CENTRAL   
24                       NEOPLASIA_MALIGNA_DO_ESÔFAGO   
25    OUTRAS_NEOPLASIAS_MALIGNAS_DE_ÓRGÃOS_DIGESTIVOS   
26                   DOENÇA_DIVERTICULAR_DO_INTESTINO   
27                                   FRATURA_DO_FÊMUR   
28  NEOPL_MALIG_FÍGADO_E_VIAS_BILIARES_INTRA-HEPÁT...   
29                          NEOPLASIA_MALIGNA_DA_MAMA   
30     PANCREATITE_AGUDA_E_OUTRAS_DOENÇAS_DO_PÂNCREAS   
31        VEIAS_VARICOSAS_DAS_EXTREMIDADES_INFERIORES   
32                        NEOPLASIA_MALIGNA_DA_BEXIGA   
33       OUTRAS_NEOPLASIAS_MALIGNAS_DO_TRATO_URINÁRIO   
34                         INFARTO_AGUDO_DO_MIOCÁRDIO   
35  ILEO_PARALÍTICO_E_OBSTRUÇÃO_INTESTINAL_SEM_HÉRNIA   
36       OUTROS_TRANSTORNOS_MENTAIS_E_COMPORTAMENTAIS   
37                      NEOPLASIA_MALIGNA_DO_ESTÔMAGO   
38                                 OUTRAS_DORSOPATIAS   
39                                LINFOMA_NÃO-HODGKIN   
40   OUTRAS_NEOPL_MALIG_ÓRG_RESPIRAT_E_INTRATORÁCICOS   
41       MENINGITE_BACTERIANA_NÃO_CLASSIF_OUTRA_PARTE   
42                                         UROLITÍASE   
43                     OUTROS_TRANSTRONOS_ARTICULARES   
44     OUTRAS_DOENÇAS_DO_NARIZ_E_DOS_SEIOS_PARANASAIS   
45  EFEITOS_CORPO_ESTRANHO_ATRAVÉS_DE_ORIFÍCIO_NAT...   
46  OUTRAS_NEOPL_MALIG_TECIDOS_LINFÓID_HEMATOP_E_R...   
47   OUTRAS_NEOPL_MALIGNAS_ÓRGÃOS_GENITAIS_MASCULINOS   
48                             INSUFICIÊNCIA_CARDÍACA   
49  NEOPLASIA_MALIGNA_DO_LÁBIO_CAVIDADE_ORAL_E_FAR...   

    Correlação com suicidio        P value  Qtd Municipios  
0                  0.422815  6.807913e-221            5113  
1                  0.412652  8.880540e-209            5096  
2                  0.407627  4.439103e-201            5040  
3                  0.403773  5.851281e-187            4781  
4                  0.391823  3.067493e-181            4947  
5                  0.391497  1.579413e-195            5351  
6                  0.381351  7.796045e-186            5381  
7                  0.380755  1.343290e-165            4805  
8                  0.376219  1.487533e-144            4297  
9                  0.372285  4.552244e-176            5367  
10                 0.371560  2.167098e-166            5090  
11                 0.370952  1.535453e-140            4305  
12                 0.367861  4.037056e-156            4876  
13                 0.365283  6.454258e-158            5008  
14                 0.364733  8.971331e-169            5374  
15                 0.363297  2.552738e-129            4138  
16                 0.361070  2.225167e-165            5380  
17                 0.361047  2.904108e-1

Save for Dashboard use

In [13]:
corrs_df.to_csv(root + "Dashboard/Spearman/correlation.csv")
corrs_df.to_csv(root + "CSV/Importance/spearman_importance.csv")